In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose= mp.solutions.pose

1.Make detections

Determining Joints


#Calculate angles

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    return angle

In [10]:
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            

In [11]:
hip,knee,ankle

([0.6312652230262756, 0.8198847770690918],
 [0.5262289047241211, 0.6179972290992737],
 [0.41397926211357117, 0.78093421459198])

In [20]:
calculate_angle(hip,knee,ankle)

57.55965998707296

In [13]:
np.multiply(knee,[640,480]).astype(int)

array([336, 296])

In [14]:
landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

x: 0.5262289047241211
y: 0.6179972290992737
z: -0.14157964289188385
visibility: 0.985906183719635

In [3]:
import time

In [4]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("KneeBendVideo.mp4")
# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [854, 640]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            if angle > 140:
                stage = "Bend your leg"
            if angle < 95 and stage == "Bend your leg":
                stage="Very good, now hold it for 8 secs"
                
                timer_sec = 8
                for i in range(timer_sec):
                    print(str(timer_sec-i) + "seconds remaining")
                    time.sleep(1)
                else:
                    print('ended loop')
                
                
                
               
                counter +=1
                print(counter)
                
                                    
        except:
            pass
        
       
        
        # Rep data
        cv2.putText(image, 'REPS:', (30,90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (80,95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
      
        cv2.putText(image, stage, 
                    (20,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
1
8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
2
8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
3
8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
4
8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
5
8seconds remaining
7seconds remaining
6seconds remaining
5seconds remaining
4seconds remaining
3seconds remaining
2seconds remaining
1seconds remaining
ended loop
6
8seconds r

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
